In [2]:
import pandas as pd
import numpy as np
from graph_mir import *
from tqdm import tqdm

c:\Users\Justin\Documents\JKU\2024WS\multimedia-search-and-retrieval\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define feature files
feature_files = {
    'mfcc_bow': 'dataset/id_mfcc_bow_mmsr.tsv',
    #'mfcc_stats': 'dataset/id_mfcc_stats_mmsr.tsv',

    #'blf_correlation' : 'dataset/id_blf_correlation_mmsr.tsv',
    #'blf_spectral': 'dataset/id_blf_spectral_mmsr.tsv',
    #'blf_spectralcontrast': 'dataset/id_blf_spectralcontrast_mmsr.tsv',
    #'blf_deltaspectral': 'dataset/id_blf_deltaspectral_mmsr.tsv',
    #'blf_logfluc': 'dataset/id_blf_logfluc_mmsr.tsv',
    #'blf_vardeltaspectral': 'dataset/id_blf_vardeltaspectral_mmsr.tsv',

    'musicnn': 'dataset/id_musicnn_mmsr.tsv',

    'lyrics_bert': 'dataset/id_lyrics_bert_mmsr.tsv',
    #'lyrics_tf-idf': 'dataset/id_lyrics_tf-idf_mmsr.tsv',
    #'lyrics_word2vec': 'dataset/id_lyrics_word2vec_mmsr.tsv',

    'resnet': 'dataset/id_resnet_mmsr.tsv',
    #'vgg': 'dataset/id_vgg19_mmsr.tsv'
    #'incp': 'dataset/id_incp_mmsr.tsv'
}

# Initialize and load the music retrieval system
music_system = MusicRetrievalSystemGraph(feature_files)
music_system.load_features()

# Create similarity graph
music_system.create_similarity_graph(similarity_threshold=0.2)

c:\Users\Justin\Documents\JKU\2024WS\multimedia-search-and-retrieval\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
Creating graph: 100%|██████████| 5148/5148 [01:56<00:00, 44.33it/s]


In [4]:
infos = pd.read_csv("dataset/id_information_mmsr.tsv", sep="\t")

song_ids = infos['id'].tolist()

similarity_matrix = np.zeros((len(song_ids), len(song_ids)))

# Create a dictionary to map song IDs to their respective index in the matrix
song_id_to_index = {song_id: idx for idx, song_id in enumerate(song_ids)}

# Populate the similarity matrix
for idx, song in tqdm(infos.iterrows(), total=len(infos)):
    reference_song_id = song['id']
    reference_index = song_id_to_index[reference_song_id]
    
    # Find the similar songs
    similar_songs = music_system.recommend_similar_tracks(reference_song_id, top_k=10)
    
    for i, similar_song_id in enumerate(similar_songs):
        if similar_song_id in song_id_to_index:
            similar_index = song_id_to_index[similar_song_id]
            similarity_matrix[reference_index, similar_index] = 10 - i

np.savetxt("./predictions/rets_graph_early_fusion_matrix_10.csv", similarity_matrix, delimiter=",")
similarity_matrix.sum()


  0%|          | 0/5148 [00:00<?, ?it/s]

100%|██████████| 5148/5148 [00:33<00:00, 152.29it/s]


283140.0

In [5]:
#infos = pd.read_csv("dataset/id_information_mmsr.tsv", sep="\t")
#
#song_ids = infos['id'].tolist()
#
#similarity_matrix = np.zeros((len(song_ids), len(song_ids)))
#
## Create a dictionary to map song IDs to their respective index in the matrix
#song_id_to_index = {song_id: idx for idx, song_id in enumerate(song_ids)}
#
## Populate the similarity matrix
#for idx, song in tqdm(infos.iterrows(), total=len(infos)):
#    reference_song_id = song['id']
#    reference_index = song_id_to_index[reference_song_id]
#    
#    # Find the similar songs
#    similar_songs = music_system.personalized_recommendation(reference_song_id, top_k=10)
#    
#    for i, similar_song_id in enumerate(similar_songs):
#        if similar_song_id[0] in song_id_to_index:
#            similar_index = song_id_to_index[similar_song_id[0]]
#            similarity_matrix[reference_index, similar_index] = 10 - i
#
#np.savetxt("y.csv", similarity_matrix, delimiter="\t")